In [2]:
# print("Hello world")
import sys
print(sys.executable)

e:\ragproject\.venv\Scripts\python.exe


In [3]:
import os
import requests
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_groq import ChatGroq

e:\ragproject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# STEP 0: Load environment variables

load_dotenv()

# Get API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Check if key exists
if not GROQ_API_KEY:
    raise ValueError("❌ GROQ_API_KEY not found in .env file")

# Initialize Groq LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

print("✅ Groq LLM initialized successfully!")

✅ Groq LLM initialized successfully!


### RAG has TWO phases — never mix them
1️⃣ Ingestion phase (slow, run ONCE)
Load PDFs, Chunk, Embed, Save vector store

2️⃣ Query phase (fast, run MANY times)
Load FAISS / pgvector & Ask questions

In [ ]:
# STEP 1: Data ingestion - loading PDFs and chunking them (RUN ONLY ONCE)
# Load PDFs → chunk → produce `chunks`

pdf_folder = r"E:\ragproject\Pdf_files"

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = []

for file in os.listdir(pdf_folder):
    if not file.lower().endswith(".pdf"):
        continue

    full_path = os.path.join(pdf_folder, file)
    print(f"\n📄 Loading {file}")

    loader = PyMuPDFLoader(full_path)

    for i, doc in enumerate(loader.lazy_load(), start=1):
        # keep metadata
        doc.metadata["source"] = file
        doc.metadata["page"] = i

        chunks.extend(splitter.split_documents([doc]))

        if i % 200 == 0:
            print(f"  {file}: {i} pages processed | chunks: {len(chunks)}")

print(f"\n✅ Finished chunking ALL PDFs")
print(f"📦 Total chunks created: {len(chunks)}")


📄 Loading postgresql-10-A4.pdf
  postgresql-10-A4.pdf: 200 pages processed | chunks: 839
  postgresql-10-A4.pdf: 400 pages processed | chunks: 1464
  postgresql-10-A4.pdf: 600 pages processed | chunks: 2263
  postgresql-10-A4.pdf: 800 pages processed | chunks: 3031
  postgresql-10-A4.pdf: 1000 pages processed | chunks: 3591
  postgresql-10-A4.pdf: 1200 pages processed | chunks: 4288
  postgresql-10-A4.pdf: 1400 pages processed | chunks: 4798
  postgresql-10-A4.pdf: 1600 pages processed | chunks: 5378
  postgresql-10-A4.pdf: 1800 pages processed | chunks: 5873
  postgresql-10-A4.pdf: 2000 pages processed | chunks: 6458
  postgresql-10-A4.pdf: 2200 pages processed | chunks: 7187
  postgresql-10-A4.pdf: 2400 pages processed | chunks: 7799

📄 Loading postgresql-11-A4.pdf
  postgresql-11-A4.pdf: 200 pages processed | chunks: 9140
  postgresql-11-A4.pdf: 400 pages processed | chunks: 9743
  postgresql-11-A4.pdf: 600 pages processed | chunks: 10515
  postgresql-11-A4.pdf: 800 pages processed

In [ ]:
# STEP 2: CREATE EMBEDDINGS (RUN ONLY ONCE)
# chunks → FAISS → save_local()
# Choose the right embedding model (fast + good)


embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

# Create FAISS index in BATCHES (IMPORTANT)

batch_size = 512
vectorstore = None
total = len(chunks)

for i in range(0, total, batch_size):
    batch = chunks[i:i + batch_size]

    if vectorstore is None:
        vectorstore = FAISS.from_documents(batch, embeddings)
    else:
        vectorstore.add_documents(batch)

    print(f"Embedded {min(i + batch_size, total)} / {total}")

vectorstore.save_local("faiss_pg_10_11")

print("✅ FAISS index saved successfully")

# This is the hardest and slowest step in RAG 
# (And a new folder: `faiss_pg_10_11` should be created in your project directory)

e:\ragproject\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajith\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For be

Embedded 512 / 16995
Embedded 1024 / 16995
Embedded 1536 / 16995
Embedded 2048 / 16995
Embedded 2560 / 16995
Embedded 3072 / 16995
Embedded 3584 / 16995
Embedded 4096 / 16995
Embedded 4608 / 16995
Embedded 5120 / 16995
Embedded 5632 / 16995
Embedded 6144 / 16995
Embedded 6656 / 16995
Embedded 7168 / 16995
Embedded 7680 / 16995
Embedded 8192 / 16995
Embedded 8704 / 16995
Embedded 9216 / 16995
Embedded 9728 / 16995
Embedded 10240 / 16995
Embedded 10752 / 16995
Embedded 11264 / 16995
Embedded 11776 / 16995
Embedded 12288 / 16995
Embedded 12800 / 16995
Embedded 13312 / 16995
Embedded 13824 / 16995
Embedded 14336 / 16995
Embedded 14848 / 16995
Embedded 15360 / 16995
Embedded 15872 / 16995
Embedded 16384 / 16995
Embedded 16896 / 16995
Embedded 16995 / 16995
✅ FAISS index saved successfully


In [45]:
# Embedding Dimensions 
print("FAISS dimension:", vectorstore.index.d)

FAISS dimension: 384


### Restart Kernel
This frees RAM and avoids accidental re-runs.

### STAGE 1: Single-stage RAG (Retrieve → Generate) 

In [ ]:
# ONLY load the FAISS index (This is now default starting point for RAG)

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

vectorstore = FAISS.load_local(
    "faiss_pg_10_11",
    embeddings,
    allow_dangerous_deserialization=True
)

print("✅ FAISS loaded, ready for queries")

✅ FAISS loaded, ready for queries


In [5]:
# Quick Retrieval Test before we connect the LLM (to make sure everything works)

query = "What is the purpose of this project?"
docs = vectorstore.similarity_search(query, k=3)
print(f"Found {len(docs)} documents for query: '{query}'")

for i, doc in enumerate(docs, 1):
    print(f"\nDocument {i}:")
    print(doc.page_content[:200])  # Show first 200 characters of each document

Found 3 documents for query: 'What is the purpose of this project?'

Document 1:
CREATE PUBLICATION ................................................................................. 1480
CREATE ROLE ..................................................................................

Document 2:
The target schema is determined by the schema parameter in the control file if that is given, other-
wise by the SCHEMA option of CREATE EXTENSION if that is given, otherwise the current default
objec

Document 3:
CREATE POLICY ................................................................................................... 1474
CREATE PUBLICATION ..............................................................


In [ ]:
docs = vectorstore.similarity_search(
    "What is logical replication in PostgreSQL 11?",
    k=3
)

for d in docs:
    print("SOURCE:", d.metadata.get("source"), "| PAGE:", d.metadata.get("page"))
    print(d.page_content[:300])
    print("-" * 50)

SOURCE: postgresql-10-A4.pdf | PAGE: 758
Chapter 31. Logical Replication
Logical replication is a method of replicating data objects and their changes, based upon their replica-
tion identity (usually a primary key). We use the term logical in contrast to physical replication, which
uses exact block addresses and byte-by-byte replication. 
--------------------------------------------------
SOURCE: postgresql-11-A4.pdf | PAGE: 799
Chapter 31. Logical Replication
Logical replication is a method of replicating data objects and their changes, based upon their replica-
tion identity (usually a primary key). We use the term logical in contrast to physical replication, which
uses exact block addresses and byte-by-byte replication. 
--------------------------------------------------
SOURCE: postgresql-10-A4.pdf | PAGE: 680
can be quickly made the new master database server. This can be synchronous or asynchronous
and can only be done for the entire database server.
A standby server can be impl

In [28]:
# Answer Generation

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key=GROQ_API_KEY
)



def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

context = format_docs(docs)

prompt = f"""
You are a PostgreSQL expert.

Answer ONLY using the context below.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
What is logical replication in PostgreSQL 11?
"""

response = llm.invoke(prompt)

print(response.content)

# If this looks good → RAG backbone is solid.

Logical replication is a method of replicating data objects and their changes, based upon their replication identity (usually a primary key). It uses a publish and subscribe model with one or more subscribers subscribing to one or more publications on a publisher node.


Till now we can call this as Vanilla RAG / Single-pass Retrieval-Augmented Generation

### ## Manual self-corrective RAG 

In [30]:
# EVALUATE ANSWER + FINAL DECISION

eval_prompt = f"""
You are an evaluator.

Question:
What is logical replication in PostgreSQL 11?

Answer:
{response.content}

Context:
{context}

Rate the answer from 0 to 1 based on:
- factual correctness
- completeness
- grounding in context

Respond ONLY with a number.
"""

eval_response = llm.invoke(eval_prompt)
print("Evaluation Score:", eval_response.content)


# FINAL THRESHOLD STRATEGY

try:
    score = float(eval_response.content.strip())
except ValueError:
    score = 0.0

if score >= 0.7:
    print("\n✅ Final Decision: ACCEPT ANSWER")
    final_answer = response.content

elif 0.3 <= score < 0.7:
    print("\n⚠️ Final Decision: NEEDS REFINEMENT")
    final_answer = (
        "The answer may be partially supported by the documents. "
        "Additional retrieval or clarification is recommended."
    )

else:
    print("\n❌ Final Decision: NOT IN DOCUMENTATION")
    final_answer = "I don't have enough information from the documents."

print("\n📌 Final Answer:\n")
print(final_answer)

Evaluation Score: 1

✅ Final Decision: ACCEPT ANSWER

📌 Final Answer:

Logical replication is a method of replicating data objects and their changes, based upon their replication identity (usually a primary key). It uses a publish and subscribe model with one or more subscribers subscribing to one or more publications on a publisher node.


### STAGE 2: Multistage RAG / Self-Corrective RAG
The system reasons, retrieves, evaluates, and possibly retrieves again before answering

The LLM critiques its own answer and fixes it

In [ ]:
# rag_chain = (
#     {
#         "context": retriever | format_docs,
#         "question": RunnablePassthrough(),
#     }
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# print("🚀 RAG System Ready!")

In [32]:
from typing import TypedDict, List
from langchain_core.documents import Document

class RAGState(TypedDict):
    question: str
    docs: List[Document]
    answer: str
    score: float
    retry_count: int

### 🧩 STEP 1 — Define LangGraph Nodes
Each node = one logical stage (same as manual code).

In [34]:
# Node 1: Retrieve

def retrieve(state: RAGState):
    docs = vectorstore.similarity_search(state["question"], k=4)
    return {"docs": docs}

In [35]:
# Node 2: Generate Answer

def generate(state: RAGState):
    context = format_docs(state["docs"])

    prompt = f"""
You are a PostgreSQL expert.

Answer ONLY using the context below.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{state["question"]}
"""

    answer = llm.invoke(prompt).content
    return {"answer": answer}

In [36]:
# Node 3: Evaluate Answer (LLM-as-Critic)

import re

def evaluate(state: RAGState):
    context = format_docs(state["docs"])

    eval_prompt = f"""
You are an evaluator.

Question:
{state["question"]}

Answer:
{state["answer"]}

Context:
{context}

Rate the answer from 0 to 1 based on:
- factual correctness
- completeness
- grounding in context

Respond ONLY with a number.
"""

    response = llm.invoke(eval_prompt).content.strip()
    match = re.search(r"\d*\.?\d+", response)
    score = float(match.group()) if match else 0.0

    return {"score": score}

In [37]:
# Node 4: Refine Query (Retry Once)

def refine(state: RAGState):
    refine_prompt = f"""
Rewrite this question to retrieve better PostgreSQL documentation.

Original question:
{state["question"]}
"""

    new_question = llm.invoke(refine_prompt).content.strip()

    return {
        "question": new_question,
        "retry_count": state["retry_count"] + 1
    }

### 🧠 STEP 2 — Decision Logic (Your Threshold Strategy)
This is exactly what you already designed.

In [38]:
from langgraph.graph import END

def decide(state: RAGState):
    score = state["score"]
    retry_count = state["retry_count"]

    if score >= 0.7:
        return "accept"

    if 0.3 <= score < 0.7 and retry_count < 1:
        return "refine"

    return "reject"

### 🧱 STEP 3 — Build the LangGraph

In [39]:
from langgraph.graph import StateGraph

graph = StateGraph(RAGState)

graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate)
graph.add_node("evaluate", evaluate)
graph.add_node("refine", refine)

graph.set_entry_point("retrieve")

graph.add_edge("retrieve", "generate")
graph.add_edge("generate", "evaluate")

graph.add_conditional_edges(
    "evaluate",
    decide,
    {
        "accept": END,
        "refine": "refine",
        "reject": END
    }
)

graph.add_edge("refine", "retrieve")

rag_graph = graph.compile()

### 🚀 STEP 4 — Run the LangGraph RAG

In [42]:
result = rag_graph.invoke({
    "question": "How does logical replication differ from physical replication in PostgreSQL 11?",
    "retry_count": 0
})

print("FINAL ANSWER:", result.get("answer"))
print("FINAL SCORE:", result.get("score"))

FINAL ANSWER: Logical replication in PostgreSQL differs from physical replication in that it uses exact block addresses and byte-by-byte replication, whereas logical replication is based upon their replication identity (usually a primary key).
FINAL SCORE: 0.9


Basic RAG	✅

Manual self-corrective RAG	✅

LangGraph self-corrective RAG	✅

Confidence-based routing	✅

Retry with refinement	✅

Hallucination rejection	✅

Production-grade control flow	✅

I want to do 
1. Scale this LangGraph to all PDFs - Time 
2. Wrap this graph in FastAPI
3. Move FAISS → PostgreSQL pgvector